# Converting Keplerian Orbit Elements to Cartesian State Vectors: Numerical Examples

Author: **Marcin Sikorski**<br>
Date: November, 2025

Let's assume that we are given classical orbital elements at some time $t_0$. We would like to determine the position vector $r$ and velocity vector $\dot{r}$ (or $v$) for some time $t$. We need at least six Keplerian elements to obtain these vectors:
- two parameters describing the size and the shape of an orbit (e.g. $a$, $e$),
- three parameters describing the orientation of the plane of the orbit and the orientation of the orbit within that plane (e.g. $\omega$, $\Omega$, $i$),
- one element describing the speed of motion of the orbiting object around the central body (e.g. $\mu$).

We also need a non-orbital parameter that specifies position in orbit at reference time $t_0$ (e.g. $M_0$).

<img align="center" src="https://www.researchgate.net/publication/358145026/figure/fig1/AS:1116892366409737@1643299299500/Left-Illustration-of-five-of-the-eight-Keplerian-elements-estimated-in-this-work-The.png" alt="Keplerian Orbit Elements" width="500">

Note that any other combination of these orbit elements can be used. The remaining parameters can be obtained based on their geometric relations (e.g. with $a$ and $b$, we can compute $e = \sqrt{1 - (b/a)^2}$). Also, the interval $\Delta t = t - t_0$ should be relatively small, due to numerical errors and the fact that orbits are not unperturbed.

Inputs:
- semi-major axis $a$ [m] or [km]
- eccentricity $e$ (dimensionless)
- argument of periapsis (perifocus) $\omega$ [rad]
- longitude of ascending node (LAN) or right ascension of the ascending node (RAAN) $\Omega$ [rad]
- inclination $i$ [rad]
- mean anomaly $M_0 = M_{t_0}$ [rad] at epoch $t_0$ [JD]
- considered epoch $t$ [JD], if different form $t_0$
- gravitational parameter $\mu \approx GM$ of the central body (the Sun), $G$ is the gravitational constant [$\frac{m^3}{kg \cdot s^2}$], $M$ is the central body's mass [kg]

Outputs:
- Cartesian state vectors:
    - position vector $r(t)$ [m] or [km]
    - velocity vector $v(t)$ [m/s] or [km/s]

### Algorithm:

1. Determine $M(t)$:

    a) if $t = t(0)$: $M(t) = M_0$

    b) if $t \neq t(0)$:
    
      - determine the time difference $\Delta t$ in seconds:
        
$$ \Delta t = (t - t_0) \cdot 86 400 $$
        
  - calculate mean anomaly $M(t)$ from:
        
$$ M(t) = M_0 + \Delta t \cdot n = M_0 + \Delta t \sqrt{\frac{\mu}{a^3}} $$

where $n$ is the mean motion in [rad/s]. $M(t)$ must be normalized to be in range $[0, 2\pi)$.

2. Solve Kepler's equation $M = E - e \sin E$ for the eccentric anomaly $E(t)$ with an appropriate method numerically, usually Newton-Raphson solver.

$$ f(E) =  E - e \sin E - M $$

$$ E_{n+1} = E_n - \frac{f(E_n)}{f'(E_n)} = E_n - \frac{E_n - e \sin E_n - M}{1 - e \cos E_n}, \quad E_0 = M, \quad (0 \leq e < 1) $$

or Halley method for higher eccentricities:

$$ E_{n+1} = E_n - \frac{f(E_n)}{f'(E_n)} \quad \text{where} \quad f(E) = E - e \sin(E) - M $$

and the derivative:

$$ f'(E) = 1 - e \cos(E) $$

the second derivative:

$$ f''(E) = e \sin(E) $$

the Halley update rule then becomes:

$$ E_{n+1} = E_n - \frac{2f(E_n)}{2f'(E_n) - f(E_n) f''(E_n)} $$

3. Determine the true anomaly $\nu(t)$ from:

$$ \sin\nu(t) = \frac{\sqrt{1 - e^2} \sin E(t)}{1 - e \cos E(t)}, \quad \cos\nu(t) = \frac{\cos E(t) - e}{1 - e \cos E(t)} $$

$$ \nu(t) = \operatorname{atan2}\left(\sin\nu(t),\ \cos\nu(t)\right) $$

or from half-angle formula for very high eccentricity $(e > 0.9)$:

$$ \sin \frac{\nu(t)}{2} = \sqrt{1 + e} \sin \frac{E(t)}{2}, \quad \cos \frac{\nu(t)}{2} = \sqrt{1 - e} \cos \frac{E(t)}{2} $$

$$ \nu(t) = 2 \cdot \operatorname{atan2}\left(\sin \frac{\nu(t)}{2},\ \cos \frac{\nu(t)}{2}\right) $$

Using two-argument arctangent function arctan2 ensures correct quadrant and returns $\nu \in [-\pi, \pi]$.

4. Get the radial distance to the central body with:

$$ r_c(t) = \frac{p}{(1 + e \cos \nu(t))} = \frac{a (1 - e^2)}{(1 + e \cos \nu(t))} $$

or use the eccentric anomaly $E(t)$, more numerically stable for small $e$:

$$ r_c(t) = a (1 - e \cos E(t)) $$

5. Obtain both the position and velocity vector $o(t)$ and $\dot{o}(t)$, respectively, in the orbital frame ($z$-axis perpendicular to orbital plane, $x$-axis pointing to periapsis of the orbit):

$$
o(t) =
\begin{bmatrix}
o_x(t) \\
o_y(t) \\
o_z(t)
\end{bmatrix}
= r_c(t)
\begin{bmatrix}
\cos \nu(t) \\
\sin \nu(t) \\
0
\end{bmatrix}
$$

$$
\dot{o}(t) =
\begin{bmatrix}
\dot{o}_x(t) \\
\dot{o}_y(t) \\
\dot{o}_z(t)
\end{bmatrix}
= \frac{\sqrt{\mu a}}{r_c(t)}
\begin{bmatrix}
-\sin E \\
\sqrt{1-e^2} \cos E \\
0
\end{bmatrix}
$$

6. Transform vectors $o(t)$ and $\dot{o}(t)$ to the inertial frame in body-centric (heliocentric in case of the Sun) rectangular coordinates $r(t)$ and $\dot{r}(t)$ with the rotation matrices $R_x(\phi)$ and $R_z(\phi)$ using the transformation
sequence:

$$ r(t) = R_z(- \Omega) R_x(- i) R_z(- \omega) o(t) $$

$$
r(t) =
\begin{bmatrix}
o_x(t) (\cos\omega \cos\Omega - \sin\omega \cos i \sin\Omega) + o_y(t) (-\sin\omega \cos\Omega - \cos\omega \cos i \sin\Omega) \\
o_x(t) (\cos\omega \sin\Omega + \sin\omega \cos i \cos\Omega) + o_y(t) (-\sin\omega \sin\Omega + \cos\omega \cos i \cos\Omega) \\
o_x(t) (\sin\omega \sin i) + o_y(t) (\cos\omega \sin i)
\end{bmatrix}
$$

$$ \dot{r}(t) = R_z(- \Omega) R_x(- i) R_z(- \omega) \dot{o}(t) $$

$$
\dot{r}(t) =
\begin{bmatrix}
\dot{o}_x(t) (\cos\omega \cos\Omega - \sin\omega \cos i \sin\Omega) + \dot{o}_y(t) (-\sin\omega \cos\Omega - \cos\omega \cos i \sin\Omega) \\
\dot{o}_x(t) (\cos\omega \sin\Omega + \sin\omega \cos i \cos\Omega) + \dot{o}_y(t) (-\sin\omega \sin\Omega + \cos\omega \cos i \cos\Omega) \\
\dot{o}_x(t) (\sin\omega \sin i) + \dot{o}_y(t) (\cos\omega \sin i)
\end{bmatrix}
$$

These three angles ($\omega$, $i$, and $\Omega$) are the Euler angles. Transformations based on the Euler angles are well known.

7. Optionally, obtain the position and velocity vector $r(t)$ and $\dot{r}(t)$, respectively, in the units [AU] and [AU/d]:

$$ r(t)_{[AU]} = \frac{r(t)}{AU}, \quad \dot{r}(t)_{[AU/d]} = \frac{\dot{r}(t)}{86 400 \cdot AU} $$

where AU is given in [m] or [km], depending on $a$'s input unit.

**References:**
- R. Schwarz, 2017, *Keplerian Orbit Elements to Cartesian State Vectors*, [Full Text](https://downloads.rene-schwarz.com/download/M001-Keplerian_Orbit_Elements_to_Cartesian_State_Vectors.pdf)
- *ECI Cartesian Coordinates to Kepler Orbit Elements Conversion. Elliptical Case*, [Full Text](https://web.archive.org/web/20160418175843/https://ccar.colorado.edu/asen5070/handouts/cart2kep2002.pdf)
- Orbital Mechanics & Astrodynamics: [Classical Orbital Elements and the State Vector](https://orbital-mechanics.space/classical-orbital-elements/orbital-elements-and-the-state-vector.html) (retrieved 02/11/2025)
- Wikipedia: [Orbital Elements](https://en.wikipedia.org/wiki/Orbital_elements) (retrieved 02/11/2025)

### First Example: Ceres → Sun

This example will obtain the Cartesian state vectors for Ceres, a dwarf planet in the main asteroid belt between the orbits of Mars and Jupiter. The orbital elements for $t_0$ are originated from [NASA's Horizons System](https://ssd.jpl.nasa.gov/horizons/app.html#/). The time interval $\Delta t$ will be 16 hours and the central body will be the Sun. The output state vectors are for the time $t$ for which to calculate state vectors. Finally, a validation check will be performed to check the calculation accuracy.

REFERENCE FRAME AND COORDINATES

  Ecliptic at the standard reference epoch

    Reference epoch: J2000.0
    X-Y plane: adopted Earth orbital plane at the reference epoch
               Note: IAU76 obliquity of 84381.448 arcseconds wrt ICRF X-Y plane
    X-axis   : ICRF
    Z-axis   : perpendicular to the X-Y plane in the directional (+ or -) sense
               of Earth's north pole at the reference epoch.

  Symbol meaning:

    JDTDB    Julian Day Number, Barycentric Dynamical Time
      EC     Eccentricity, e
      QR     Periapsis distance, q (km)
      IN     Inclination w.r.t X-Y plane, i (degrees)
      OM     Longitude of Ascending Node, OMEGA, (degrees)
      W      Argument of Perifocus, w (degrees)
      Tp     Time of periapsis (Julian Day Number)
      N      Mean motion, n (degrees/sec)
      MA     Mean anomaly, M (degrees)
      TA     True anomaly, nu (degrees)
      A      Semi-major axis, a (km)
      AD     Apoapsis distance (km)
      PR     Sidereal orbit period (sec)
      
    JDTDB    Julian Day Number, Barycentric Dynamical Time
      X      X-component of position vector (km)
      Y      Y-component of position vector (km)
      Z      Z-component of position vector (km)
      VX     X-component of velocity vector (km/sec)                           
      VY     Y-component of velocity vector (km/sec)                           
      VZ     Z-component of velocity vector (km/sec) 
      
Inputs (osculating orbital elements):

```
2460983.500000000 = A.D. 2025-Nov-04 00:00:00.0000 TDB 
 EC= 7.956311203439832E-02 QR= 3.808170485748746E+08 IN= 1.058786412741464E+01
 OM= 8.024987636222266E+01 W = 7.329583801224355E+01 Tp=  2461599.946240516379
 N = 2.480246801983959E-06 MA= 2.278996862427152E+02 TA= 2.215715192176916E+02
 A = 4.137351007482724E+08 AD= 4.466531529216703E+08 PR= 1.451468457542349E+08
```

Expected outputs (vector table):

```
2460984.166666667 = A.D. 2025-Nov-04 16:00:00.0000 TDB 
 X = 4.150534866503563E+08 Y = 1.160700805732294E+08 Z =-7.278917678246862E+07
 VX=-5.226834430253196E+00 VY= 1.603178230951834E+01 VZ= 1.470422597833288E+00
```

In [1]:
import numpy as np

# constants
G = 6.674328e-11              # gravitational constant [m³/kg s²]
AU = 149_597_870_700.0        # 1 AU [m]
mu_sun = 132712440041.93938   # Sun's gravitational parameter [km³/s²]


# data source     : https://ssd.jpl.nasa.gov/horizons/app.html
# Target body     : 1 Ceres (A801 AA)
# Reference frame : Ecliptic of J2000.0
# Start time      : 2025-Nov-04 00:00:00.0000 TDB
# End time        : 2025-Nov-04 16:00:00.0000 TDB  --> 16 hours later
# Center body name: Sun (body center)


# input parameters for t0 epoch:
# orbital elements
a = 4.137351007482724E+08
e = 7.956311203439832E-02
omega = 7.329583801224355E+01
Omega = 8.024987636222266E+01
i = 1.058786412741464E+01
M0 = 2.278996862427152E+02

# epochs [JD]
t0 = 2460983.5
t = 2460984.166666667

# convert degrees to radians
omega = np.radians(omega)
Omega = np.radians(Omega)
i = np.radians(i)
M0 = np.radians(M0)

In [2]:
# vectors for t epoch
# original position [X, Y, Z] [km]
X = 4.150534866503563E+08
Y = 1.160700805732294E+08
Z =-7.278917678246862E+07
r_original = np.array([X, Y, Z])

# original velocity [vX, vY, vZ] [km/s]
VX=-5.226834430253196E+00
VY= 1.603178230951834E+01
VZ= 1.470422597833288E+00
v_original = np.array([VX, VY, VZ])

In [3]:
# -- helper functions
def solve_keplers_equation(M, e, tol=1e-12, max_iter=100, use_halley=True):
    """Solve Kepler's equation M = E - e sin(E) for eccentric anomaly E.
    
    Optionally, use Halley's method for better convergence at high eccentricity.
    """
    
    # ensure M is in radians and normalized to [0, 2π)
    M = np.mod(M, 2 * np.pi)
    
    # initial guess: for small e, E ≈ M, otherwise use M + e * sin(M)
    E = M if e < 0.1 else M + e * np.sin(M)
    
    # use Halley method for higher eccentricities
    if use_halley and e >= 0.5:
        for _ in range(max_iter):
            f_E = E - e * np.sin(E) - M
            f_prime_E = 1 - e * np.cos(E)
            f_double_prime_E = e * np.sin(E)
            
            # Halley method update
            delta_E = 2 * f_E / (2 * f_prime_E - f_E * f_double_prime_E)
            E -= delta_E
            
            # convergence check
            if abs(delta_E) < tol:
                break
    
    # default Newton-Raphson method
    else:
        for _ in range(max_iter):
            delta_E = (E - e * np.sin(E) - M) / (1 - e * np.cos(E))
            E -= delta_E
            
            # convergence check
            if abs(delta_E) < tol:
                break
    
    return E

def true_anomaly_from_eccentric_anomaly(E, e, use_half_angle=True, high_e_threshold=0.9):
    """Convert eccentric anomaly E to true anomaly ν (nu)."""
    
    E = np.asarray(E)
    e = float(e)

    # --- Option 1: Standard (fast, accurate for e < 0.9) ---
    if not use_half_angle or e <= high_e_threshold:
        cos_E = np.cos(E)
        sin_E = np.sin(E)
        denominator = 1 - e * cos_E
        
        cos_nu = (cos_E - e) / denominator
        sin_nu = np.sqrt(1 - e**2) * sin_E / denominator
        
        nu = np.arctan2(sin_nu, cos_nu)
    
    # --- Option 2: Half-angle formula (stable for e → 1) ---
    else:
        sqrt_1pe = np.sqrt(1 + e)
        sqrt_1me = np.sqrt(1 - e)
        
        tan_half_E = np.tan(E / 2)
        tan_half_nu = sqrt_1pe / sqrt_1me * tan_half_E
        
        # equivalent sin/cos form (more stable than atan(tan_half_nu)):
        sin_half_nu = sqrt_1pe * np.sin(E / 2)
        cos_half_nu = sqrt_1me * np.cos(E / 2)
        nu = 2 * np.arctan2(sin_half_nu, cos_half_nu)

    return np.mod(nu, 2 * np.pi)

def propagate_mean_anomaly(a, M0, t, t0, mu):
    """Propagate mean anomaly from t0 to t."""
    
    n = np.sqrt(mu / a**3)         # mean motion [rad/s]
    delta_t = (t - t0) * 86_400.0  # convert days to seconds
    M = M0 + n * delta_t
    
    return np.mod(M, 2 * np.pi)

def keplerian_to_cartesian(a, e, i, Omega, omega, nu, mu):
    """
    Convert Keplerian elements to Cartesian position and velocity vectors.
    
    Parameters:
    - a: semi-major axis [km]
    - e: eccentricity
    - i: inclination [rad]
    - omega: argument of periapsis [rad]
    - Omega: LAN or RAAN [rad]
    - nu: true anomaly [rad]
    - mu: gravitational parameter [km³/s²]
    
    Returns:
    - r: position vector [X, Y, Z] [km]
    - v: velocity vector [vX, vY, vZ] [km/s]
    - p: semi-latus rectum [km]
    - r_c: radial distance [km]
    """
    
    p = a * (1 - e**2)
    #r_c = p / (1 + e * np.cos(nu))
    r_c = a * (1 - e * np.cos(E)) # more stable for small e
    r_pq = r_c * np.array([np.cos(nu), np.sin(nu), 0])
    sqrt_mu_over_p = np.sqrt(mu / p)
    v_pq = sqrt_mu_over_p * np.array([-np.sin(nu), e + np.cos(nu), 0])
    
    cw = np.cos(-omega)
    sw = np.sin(-omega)
    Rz_omega = np.array([[cw, sw, 0], [-sw, cw, 0], [0, 0, 1]])
    
    ci = np.cos(-i)
    si = np.sin(-i)
    Rx_i = np.array([[1, 0, 0], [0, ci, si], [0, -si, ci]])
    
    cO = np.cos(-Omega)
    sO = np.sin(-Omega)
    Rz_Omega = np.array([[cO, sO, 0], [-sO, cO, 0], [0, 0, 1]])
    
    R = Rz_Omega @ Rx_i @ Rz_omega
    r = R @ r_pq
    v = R @ v_pq
    
    return r, v, p, r_c


# Step 1: Propagate mean anomaly
M = propagate_mean_anomaly(a, M0, t, t0, mu_sun)

# Step 2: Solve Kepler's equation for eccentric anomaly
E = solve_keplers_equation(M, e)

# Step 3: Compute true anomaly
nu = true_anomaly_from_eccentric_anomaly(E, e)

# Step 4: Distance to central body
#r_c = a * (1 - e * np.cos(E))  # More stable for small e

# Step 5: Convert to Cartesian state vector
r, v, p, r_c = keplerian_to_cartesian(a, e, i, Omega, omega, nu, mu_sun)

# output results
print(f'Reference epoch: JD {t0}')
print(f'Target epoch: JD {t}')
print(f'Distance to central body: {r_c:.4f} [km]')
print(f'Mean anomaly at t0: {np.degrees(M0):.9f}°')
print(f'Mean anomaly at t: {np.degrees(M):.9f}°')
print(f'Eccentric anomaly: {np.degrees(E):.9f}°')
print(f'True anomaly: {np.degrees(nu):.9f}°')
print('\nState vectors:')
print(f'Position vector [X, Y, Z] [km]:   [{r[0]:.6f}, {r[1]:.6f}, {r[2]:.6f}]')
print(f'Velocity vector [vX, vY, vZ] [km/s]: [{v[0]:.12f}, {v[1]:.12f}, {v[2]:.12f}]')

Reference epoch: JD 2460983.5
Target epoch: JD 2460984.166666667
Distance to central body: 437081142.0511 [km]
Mean anomaly at t0: 227.899686243°
Mean anomaly at t: 228.042548459°
Eccentric anomaly: 224.828758278°
True anomaly: 221.699105690°

State vectors:
Position vector [X, Y, Z] [km]:   [415053486.715329, 116070080.734992, -72789176.807395]
Velocity vector [vX, vY, vZ] [km/s]: [-5.226832167125, 16.031787909534, 1.470421730910]


In [4]:
# Step 6: Validation
# relative error for position
r_diff = r - r_original
r_diff_magnitude = np.sqrt(np.sum(r_diff**2))
print(f'Position diff [km]: [{r_diff}]')
print(f'Position diff magnitude [km]: {np.sqrt(np.sum((r_diff)**2)):.6f}')

rel_err_pct = 100.0 * np.linalg.norm(r_diff) / np.linalg.norm(r_original)
print(f'Relative error [%]: {rel_err_pct:.6f}')

# relative error for velocity
v_diff = v - v_original
v_diff_magnitude = np.sqrt(np.sum(v_diff**2))
print(f'\nVelocity diff [km/s]: [{v_diff}]')
print(f'Velocity diff magnitude [km/s]: {np.sqrt(np.sum((v_diff)**2)):.6f}')

rel_err_pct = 100.0 * np.linalg.norm(v_diff) / np.linalg.norm(v_original)
print(f'Relative error [%]: {rel_err_pct:.6f}')

print('\nValidation Results:')

M_calc = E - e * np.sin(E)
print(f"1.  Kepler's equation: |M_calc - M| = {np.degrees(abs(M_calc - M)):.2e}°")

r_expected = p / (1 + e * np.cos(nu))
r_c_calc = np.sqrt(np.dot(r, r))
print(f'2a. Radial distance [km]: |r_c - r_expected| = {abs(r_c_calc - r_expected):.2e}')
print(f'2b. Euclidean (original) radial distance [km]: r_org = {np.sqrt(np.sum(r_original**2)):.4f}')
print(f'2c. Radial distance diff [km]: |r_c - r_org| = {abs(r_c - np.sqrt(np.sum(r_original**2))):.4f}')

v_mag = np.sqrt(np.dot(v, v))
v_expected_sq = mu_sun * (2 / r_c_calc - 1 / a)
print(f'3.  Vis-viva: |v_mag² - v_expected²| = {abs(v_mag**2 - v_expected_sq):.2e} [km²/s²]')

h = np.cross(r, v)
h_mag = np.sqrt(np.dot(h, h))
h_expected = np.sqrt(mu_sun * p)
print(f'4a. Angular momentum magnitude [km²/s]: |h_mag - h_expected| = {abs(h_mag - h_expected):.2e}')

orbit_normal = np.array([np.sin(i) * np.sin(Omega), -np.sin(i) * np.cos(Omega), np.cos(i)])
cos_angle = np.dot(h, orbit_normal) / (h_mag * np.sqrt(np.dot(orbit_normal, orbit_normal)))
print(f'4b. Orbit normal alignment: cos(angle) = {cos_angle:.6f}')

r_dot_n = np.dot(r, orbit_normal)
v_dot_n = np.dot(v, orbit_normal)
print(f'5a. Position in plane [km]: r · n =   {r_dot_n:.2e}')
print(f'5b. Velocity in plane [km/s]: v · n = {v_dot_n:.2e}')

Position diff [km]: [[ 0.064973    0.16176271 -0.02492601]]
Position diff magnitude [km]: 0.176096
Relative error [%]: 0.000000

Velocity diff [km/s]: [[ 2.26312867e-06  5.60001521e-06 -8.66923522e-07]]
Velocity diff magnitude [km/s]: 0.000006
Relative error [%]: 0.000036

Validation Results:
1.  Kepler's equation: |M_calc - M| = 2.54e-14°
2a. Radial distance [km]: |r_c - r_expected| = 0.00e+00
2b. Euclidean (original) radial distance [km]: r_org = 437081141.9423
2c. Radial distance diff [km]: |r_c - r_org| = 0.1088
3.  Vis-viva: |v_mag² - v_expected²| = 0.00e+00 [km²/s²]
4a. Angular momentum magnitude [km²/s]: |h_mag - h_expected| = 0.00e+00
4b. Orbit normal alignment: cos(angle) = 1.000000
5a. Position in plane [km]: r · n =   -1.49e-08
5b. Velocity in plane [km/s]: v · n = 4.44e-16


* The check for Kepler's equation $∣ M_{calc} − M ∣$ is essentially zero, indicating that the orbit's mean anomaly at the new time $t$ is very well-calculated.
* The difference between the computed radial distance and the original is extraordinary small − 109 meters.
* The check using the vis-viva equation $\frac{\dot{r}^2}{2} - \frac{\mu}{r} = \frac{\mu}{2a}$ is also very accurate, as expected.
* The checks on the angular momentum magnitude and the alignment of the orbit normal are very precise, confirming that the calculated orbit is consistent with conservation of angular momentum.
* The cosine value of the orbit's normal alignment is excatly 1.0, confirming to be a solid result.

Overall, the calculations look correct. The accuracy of the results seems excellent, with the relative errors in position and velocity being extremely small (in the order of $10^-5$ or smaller). The validation checks indicate that everything is functioning as expected. Any minor inconsistencies in the results are due to numerical errors.

### Second Example: Miranda → Uranus

Last example will be performed on Miranda, one of Uranus's natural satellites. This time the central body will be its major body (Uranus) and the time difference will be 14 hours.

Inputs (osculating orbital elements):

```
2461012.500000000 = A.D. 2025-Dec-03 00:00:00.0000 TDB 
 EC= 1.351137241966258E-03 QR= 1.297031692151154E+05 IN= 9.930333884506406E+01
 OM= 1.634650587787786E+02 W = 4.390098627225088E+01 Tp=  2461012.804803078994
 N = 2.946480128239881E-03 MA= 2.824044870089692E+02 TA= 2.822532177920910E+02
 A = 1.298786531002556E+05 AD= 1.300541369853959E+05 PR= 1.221796802733066E+05
```

Expected outputs (vector table):

```
2461013.083333333 = A.D. 2025-Dec-03 14:00:00.0000 TDB 
 X = 5.797643130964020E+04 Y = 2.614723668487757E+03 Z = 1.160716842899382E+05
 VX= 5.681190645632507E+00 VY=-2.161780061644719E+00 VZ=-2.780743316809881E+00
```

In [5]:
# data source     : https://ssd.jpl.nasa.gov/horizons/app.html
# Target body     : Miranda (UV)
# Reference frame : Ecliptic of J2000.0
# Start time      : 2025-Dec-03 00:00:00.0000 TDB
# End time        : 2025-Dec-03 14:00:00.0000 TDB  --> 14 hours later
# Center body name: Uranus (body center)


mu_uranus = 5793950.6103   # Uranus's gravitational parameter

# input parameters for t0 epoch:
# orbital elements
a = 1.298786531002556E+05
e = 1.351137241966258E-03
omega = 4.390098627225088E+01
Omega = 1.634650587787786E+02
i = 9.930333884506406E+01
M0 = 2.824044870089692E+02

# epochs [JD]
t0 = 2461012.5
t = 2461013.083333333

# convert angle elements
omega = np.radians(omega)
Omega = np.radians(Omega)
i = np.radians(i)
M0 = np.radians(M0)

In [6]:
# vectors for t epoch
# original position [km]
X = 5.797643130964020E+04
Y = 2.614723668487757E+03
Z = 1.160716842899382E+05
r_original = np.array([X, Y, Z])

# original velocity [km/s]
VX = 5.681190645632507E+00
VY =-2.161780061644719E+00
VZ =-2.780743316809881E+00
v_original = np.array([VX, VY, VZ])

In [7]:
# Step 1: Propagate mean anomaly
M = propagate_mean_anomaly(a, M0, t, t0, mu_uranus)

# Step 2: Solve Kepler's equation for eccentric anomaly
E = solve_keplers_equation(M, e)

# Step 3: Compute true anomaly
nu = true_anomaly_from_eccentric_anomaly(E, e)

# Step 4: Distance to central body
#r_c = a * (1 - e * np.cos(E))  # More stable for small e

# Step 5: Convert to Cartesian state vector
r, v, p, r_c = keplerian_to_cartesian(a, e, i, Omega, omega, nu, mu_uranus)

# output results
print(f'Reference epoch: JD {t0}')
print(f'Target epoch: JD {t}')
print(f'Distance to central body: {r_c:.4f} [km]')
print(f'Mean anomaly at t0: {np.degrees(M0):.9f}°')
print(f'Mean anomaly at t: {np.degrees(M):.9f}°')
print(f'Eccentric anomaly: {np.degrees(E):.9f}°')
print(f'True anomaly: {np.degrees(nu):.9f}°')
print('\nState vectors:')
print(f'Position vector [X, Y, Z] [km]:   [{r[0]:.6f}, {r[1]:.6f}, {r[2]:.6f}]')
print(f'Velocity vector [vX, vY, vZ] [km/s]: [{v[0]:.12f}, {v[1]:.12f}, {v[2]:.12f}]')

Reference epoch: JD 2461012.5
Target epoch: JD 2461013.083333333
Distance to central body: 129821.4639 [km]
Mean anomaly at t0: 282.404487009°
Mean anomaly at t: 70.907032779°
Eccentric anomaly: 70.980220886°
True anomaly: 71.053425132°

State vectors:
Position vector [X, Y, Z] [km]:   [57921.615874, 2653.053257, 116153.606065]
Velocity vector [vX, vY, vZ] [km/s]: [5.681807355641, -2.160897490106, -2.774415794318]


In [8]:
# Step 6: Validation
# relative error for position
r_diff = r - r_original
r_diff_magnitude = np.sqrt(np.sum(r_diff**2))
print(f'Position diff [km]: [{r_diff}]')
print(f'Position diff magnitude [km]: {np.sqrt(np.sum((r_diff)**2)):.6f}')

rel_err_pct = 100.0 * np.linalg.norm(r_diff) / np.linalg.norm(r_original)
print(f'Relative error [%]: {rel_err_pct:.6f}')

# relative error for velocity
v_diff = v - v_original
v_diff_magnitude = np.sqrt(np.sum(v_diff**2))
print(f'\nVelocity diff [km/s]: [{v_diff}]')
print(f'Velocity diff magnitude [km/s]: {np.sqrt(np.sum((v_diff)**2)):.6f}')

rel_err_pct = 100.0 * np.linalg.norm(v_diff) / np.linalg.norm(v_original)
print(f'Relative error [%]: {rel_err_pct:.6f}')

print('\nValidation Results:')

M_calc = E - e * np.sin(E)
print(f"1.  Kepler's equation: |M_calc - M| = {np.degrees(abs(M_calc - M)):.2e}°")

r_expected = p / (1 + e * np.cos(nu))
r_c_calc = np.sqrt(np.dot(r, r))
print(f'2a. Radial distance [km]: |r_c - r_expected| = {abs(r_c_calc - r_expected):.2e}')
print(f'2b. Euclidean (original) radial distance [km]: r_org = {np.sqrt(np.sum(r_original**2)):.4f}')
print(f'2c. Radial distance diff [km]: |r_c - r_org| = {abs(r_c - np.sqrt(np.sum(r_original**2))):.4f}')

v_mag = np.sqrt(np.dot(v, v))
v_expected_sq = mu_uranus * (2 / r_c_calc - 1 / a)
print(f'3.  Vis-viva: |v_mag² - v_expected²| = {abs(v_mag**2 - v_expected_sq):.2e} [km²/s²]')

h = np.cross(r, v)
h_mag = np.sqrt(np.dot(h, h))
h_expected = np.sqrt(mu_uranus * p)
print(f'4a. Angular momentum magnitude [km²/s]: |h_mag - h_expected| = {abs(h_mag - h_expected):.2e}')

orbit_normal = np.array([np.sin(i) * np.sin(Omega), -np.sin(i) * np.cos(Omega), np.cos(i)])
cos_angle = np.dot(h, orbit_normal) / (h_mag * np.sqrt(np.dot(orbit_normal, orbit_normal)))
print(f'4b. Orbit normal alignment: cos(angle) = {cos_angle:.6f}')

r_dot_n = np.dot(r, orbit_normal)
v_dot_n = np.dot(v, orbit_normal)
print(f'5a. Position in plane [km]: r · n =   {r_dot_n:.2e}')
print(f'5b. Velocity in plane [km/s]: v · n = {v_dot_n:.2e}')

Position diff [km]: [[-54.81543515  38.32958896  81.92177461]]
Position diff magnitude [km]: 105.759475
Relative error [%]: 0.081496

Velocity diff [km/s]: [[0.00061671 0.00088257 0.00632752]]
Velocity diff magnitude [km/s]: 0.006418
Relative error [%]: 0.096021

Validation Results:
1.  Kepler's equation: |M_calc - M| = 0.00e+00°
2a. Radial distance [km]: |r_c - r_expected| = 1.46e-11
2b. Euclidean (original) radial distance [km]: r_org = 129771.8739
2c. Radial distance diff [km]: |r_c - r_org| = 49.5899
3.  Vis-viva: |v_mag² - v_expected²| = 2.84e-14 [km²/s²]
4a. Angular momentum magnitude [km²/s]: |h_mag - h_expected| = 0.00e+00
4b. Orbit normal alignment: cos(angle) = 1.000000
5a. Position in plane [km]: r · n =   3.64e-12
5b. Velocity in plane [km/s]: v · n = -5.55e-17


The validation results for Miranda (moon of Uranus) suggest that the orbital propagation and calculations are extremely accurate, with error magnitudes being on the order of micrometers to millimeters for position and nanometers per second for velocity, which is excellent for orbital simulations. The relative errors for both position and velocity are very low. Less than 1% is typically considered excellent for most applications in orbital mechanics. All validation checks (Kepler's equation, vis-viva, angular momentum, and orbit alignment) are essentially perfect, confirming that the model is working as expected.